# Explore Various Retrieval Strategies with Retrievers in LangChain

## Install OpenAI, and LangChain dependencies

In [ ]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

## Install Chroma Vector DB and LangChain wrapper

In [ ]:
!pip install langchain-chroma

## Enter Open AI API Key

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Setup Environment Variables

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [4]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

## Vector Databases

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector database takes care of storing embedded data and performing vector search for you.

### Chroma Vector DB

[Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

### Create a Vector DB and persist on disk

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [5]:
docs = [
 'Quantum mechanics describes the behavior of very small particles.',
 'Photosynthesis is the process by which green plants make food using sunlight.',
 'Artificial Intelligence aims to create machines that can think and learn.',
 'The pyramids of Egypt are historical monuments that have stood for thousands of years.',
 'New Delhi is the capital of India and the seat of all three branches of the Government of India.',
 'Biology is the study of living organisms and their interactions with the environment.',
 'Music therapy can aid in the mental well-being of individuals.',
 'Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.',
 'The Milky Way is just one of billions of galaxies in the universe.',
 'Economic theories help understand the distribution of resources in society.',
 'Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.',
 'Yoga is an ancient practice that involves physical postures and meditation.'
]

In [6]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes 1 min on Colab
chroma_db = Chroma.from_texts(texts=docs, collection_name='db_docs',
                              # need to set the distance function to cosine else it uses euclidean by default
                              # check https://docs.trychroma.com/guides#changing-the-distance-function
                              collection_metadata={"hnsw:space": "cosine"},
                              embedding=openai_embed_model)

## Vector Database Retrievers

Here we will explore the following retrieval strategies on our Vector Database:

- Similarity or Ranking based Retrieval
- Similarity with Threshold Retrieval
- Custom Retriever with Similarity Scores + Thresholding
- Multi Query Retrieval
- Contextual Compression Retrieval
- Ensemble Retrieval

### Similarity or Ranking based Retrieval

We use cosine similarity here and retrieve the top 3 similar documents based on the user input query

In [7]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 3})

In [8]:
query = "what is the capital of India?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

[Document(metadata={}, page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(metadata={}, page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.'),
 Document(metadata={}, page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.')]

In [9]:
query = "what is the old capital of India?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

[Document(metadata={}, page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.'),
 Document(metadata={}, page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(metadata={}, page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.')]

In [10]:
query = "how do plants make food?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

[Document(metadata={}, page_content='Photosynthesis is the process by which green plants make food using sunlight.'),
 Document(metadata={}, page_content='Biology is the study of living organisms and their interactions with the environment.'),
 Document(metadata={}, page_content='Artificial Intelligence aims to create machines that can think and learn.')]

We use maximum marginal relevance ranking here and retrieve the top 3 similar documents based on the user input query

In [11]:
mmr_retriever = chroma_db.as_retriever(search_type="mmr",
                                       search_kwargs={"k": 3,
                                                      'fetch_k': 10})

In [12]:
query = "what is the capital of India?"
top3_docs = mmr_retriever.invoke(query)
top3_docs

[Document(metadata={}, page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(metadata={}, page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.'),
 Document(metadata={}, page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.')]

In [13]:
query = "what is the old capital of India?"
top3_docs = mmr_retriever.invoke(query)
top3_docs

[Document(metadata={}, page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.'),
 Document(metadata={}, page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(metadata={}, page_content='Yoga is an ancient practice that involves physical postures and meditation.')]

In [14]:
query = "how do plants make food?"
top3_docs = mmr_retriever.invoke(query)
top3_docs

[Document(metadata={}, page_content='Photosynthesis is the process by which green plants make food using sunlight.'),
 Document(metadata={}, page_content='Artificial Intelligence aims to create machines that can think and learn.'),
 Document(metadata={}, page_content='Economic theories help understand the distribution of resources in society.')]

### Similarity with Threshold Retrieval

We use cosine similarity here and retrieve the top 3 similar documents based on the user input query and also introduce a cutoff to not return any documents which are below a certain similarity threshold

In [15]:
similarity_threshold_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                                        search_kwargs={"k": 3,
                                                                       "score_threshold": 0.3})

In [16]:
query = "what is the capital of India?"
top3_docs = similarity_threshold_retriever.invoke(query)
top3_docs

[Document(metadata={}, page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(metadata={}, page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.'),
 Document(metadata={}, page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.')]

In [17]:
query = "what is the old capital of India?"
top3_docs = similarity_threshold_retriever.invoke(query)
top3_docs

[Document(metadata={}, page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.'),
 Document(metadata={}, page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(metadata={}, page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.')]

In [18]:
query = "how do plants make food?"
top3_docs = similarity_threshold_retriever.invoke(query)
top3_docs

[Document(metadata={}, page_content='Photosynthesis is the process by which green plants make food using sunlight.')]

### Custom Retriever with Similarity Scores + Thresholding

Here we will create a custom retriever which will:

- Retrieve documents with cosine distance
- Convert to similarity score and apply thresholding
- Return topk documents above a similarity threshold

In [19]:
query = 'how do plants make food?'
chroma_db.similarity_search_with_score(query, k=3)

[(Document(metadata={}, page_content='Photosynthesis is the process by which green plants make food using sunlight.'),
  0.35380274057388306),
 (Document(metadata={}, page_content='Biology is the study of living organisms and their interactions with the environment.'),
  0.8317177891731262),
 (Document(metadata={}, page_content='Artificial Intelligence aims to create machines that can think and learn.'),
  0.8765133619308472)]

In [20]:
chroma_db._select_relevance_score_fn()

<function langchain_core.vectorstores.base.VectorStore._cosine_relevance_score_fn(distance: 'float') -> 'float'>

In [21]:
# converts cosine distance to similarity; cosine_similarity = 1 - cosine_distance
cosine_sim = chroma_db._select_relevance_score_fn()
cosine_sim(0.35375)

0.64625

In [22]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain

@chain
def custom_retriever(query: str, topk=3, threshold_score=0.3) -> List[Document]:
    # get similarity conversion function (converts cosine distance to similarity)
    cosine_sim = chroma_db._select_relevance_score_fn()
    # get topk documents with lowest cosine distance
    docs, scores = zip(*chroma_db.similarity_search_with_score(query, k=topk))
    final_docs = []
    for doc, score in zip(docs, scores):
        # convert cosine distance to similarity
        score = cosine_sim(score)
        doc.metadata["score"] = round(score, 3)
        # check if score is above threshold
        if score > threshold_score:
            final_docs.append(doc)

    return final_docs

In [23]:
query = "what is the financial capital of India?"
top3_docs = custom_retriever.invoke(query, topk=3, threshold_score=0.51)
top3_docs

[Document(metadata={'score': 0.69}, page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.'),
 Document(metadata={'score': 0.54}, page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.')]

In [24]:
query = 'how do plants make food?'
top3_docs = custom_retriever.invoke(query, topk=3, threshold_score=0.5)
top3_docs

[Document(metadata={'score': 0.646}, page_content='Photosynthesis is the process by which green plants make food using sunlight.')]

### Multi Query Retrieval

Retrieval may produce different results with subtle changes in query wording, or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.

The [`MultiQueryRetriever`](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents.

In [25]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [26]:
from langchain.retrievers.multi_query import MultiQueryRetriever
# Set logging for the queries
import logging

similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 3})

mq_retriever = MultiQueryRetriever.from_llm(
    retriever=similarity_retriever, llm=chatgpt
)

logging.basicConfig()
# so we can see what queries are generated by the LLM
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [27]:
query = "financial capital of India?"
docs = mq_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['What is the capital city of India in terms of its financial significance?  ', 'Which city in India serves as the financial hub of the country?  ', 'Can you tell me about the financial capital of India?']


[Document(metadata={}, page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.'),
 Document(metadata={}, page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(metadata={}, page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.')]

In [28]:
query = "old capital of India?"
docs = mq_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['What was the former capital of India?  ', "Which city served as India's capital before the current one?  ", 'Can you tell me the historical capital of India?']


[Document(metadata={}, page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.'),
 Document(metadata={}, page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(metadata={}, page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.')]

### Contextual Compression Retrieval

The information most relevant to a query may be buried in a document with a lot of irrelevant text. Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this. The idea is simple: instead of immediately returning retrieved documents as-is, you can compress them using the context of the given query, so that only the relevant information is returned.

This compression can happen in the form of:

- Remove parts of the content of retrieved documents which are not relevant to the query. This is done by extracting only relevant parts of the document to the given query

- Filter out documents which are not relevant to the given query but do not remove content from the document

Here we wrap our base cosine distance retriever with a `ContextualCompressionRetriever`. Then we'll add an `LLMChainExtractor`, which will iterate over the initially returned documents and extract from each only the content that is relevant to the query.

In [29]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# simple cosine distance based retriever
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 3})

# extracts from each document only the content that is relevant to the query
compressor = LLMChainExtractor.from_llm(llm=chatgpt)

# retrieves the documents similar to query and then applies the compressor
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=similarity_retriever
)

In [30]:
query = "what is the financial capital of India?"
docs = compression_retriever.invoke(query)
docs

[Document(metadata={}, page_content='Mumbai is the financial capital of India.')]

In [31]:
query = "what is the old capital of India?"
docs = compression_retriever.invoke(query)
docs

[Document(metadata={}, page_content='Calcutta served as the de facto capital of India until 1911.')]

The `LLMChainFilter` is slightly simpler but more robust compressor that uses an LLM chain to decide which of the initially retrieved documents to filter out and which ones to return, without manipulating the document contents.

In [32]:
from langchain.retrievers.document_compressors import LLMChainFilter

#  decides which of the initially retrieved documents to filter out and which ones to return
_filter = LLMChainFilter.from_llm(llm=chatgpt)

# retrieves the documents similar to query and then applies the filter
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=similarity_retriever
)

In [33]:
query = "what is the financial capital of India?"
docs = compression_retriever.invoke(query)
docs

[Document(metadata={}, page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.')]

In [34]:
query = "what is the old capital of India?"
docs = compression_retriever.invoke(query)
docs

[Document(metadata={}, page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.')]

In [35]:
query = "how do plants live?"
docs = compression_retriever.invoke(query)
docs

[Document(metadata={}, page_content='Photosynthesis is the process by which green plants make food using sunlight.')]

### Ensemble Retrieval

The `EnsembleRetriever` takes a list of retrievers as input and ensemble the results of each of their retrievals and rerank the results based on the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

In [36]:
from langchain.retrievers import EnsembleRetriever

# simple cosine distance based retriever
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 3})

# retrieves the documents similar to query and then applies the filter
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=similarity_retriever
)


ensemble_retriever = EnsembleRetriever(
    retrievers=[compression_retriever, similarity_retriever],
    weights=[0.7, 0.3]
)

In [37]:
query = "what is the financial capital of India?"
docs = ensemble_retriever.invoke(query)
docs

[Document(metadata={}, page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.'),
 Document(metadata={}, page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(metadata={}, page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.')]

In [38]:
query = "how do plants live?"
docs = ensemble_retriever.invoke(query)
docs

[Document(metadata={}, page_content='Photosynthesis is the process by which green plants make food using sunlight.'),
 Document(metadata={}, page_content='Biology is the study of living organisms and their interactions with the environment.'),
 Document(metadata={}, page_content='Music therapy can aid in the mental well-being of individuals.')]

We will cover more retrieval strategies in the Advanced Course on RAG and LangChain in the future including:

- Self Query Retrieval
- Hybrid Search Retrieval
- Parent Document Retrieval
- Reranker Retrieval

and more...